In [73]:
import tensorflow as tf
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json

In [74]:
#讀檔

num = 0
train_data = []
test_data = []
train_size = 50
test_size = 10
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

input_file = open('translation2019zh_train.json', encoding = 'utf8')
for line in input_file.readlines():
    num = num + 1
    if num > train_size + test_size:
        break
    elif num > train_size:
        test_data.append(json.loads(line))
    else:
        train_data.append(json.loads(line))
input_file.close()

'''
input_file = open('translation2019zh_valid.json', encoding = 'utf8')
for line in input_file.readlines():
    num = num + 1
    #data.append(json.loads(line))
input_file.close()'''

"\ninput_file = open('translation2019zh_valid.json', encoding = 'utf8')\nfor line in input_file.readlines():\n    num = num + 1\n    #data.append(json.loads(line))\ninput_file.close()"

In [75]:
print(len(train_data))
print(len(test_data))

50
10


In [76]:
"""#簡轉繁
from opencc import OpenCC

cc = OpenCC('s2t')

for data in train_data:
    data['chinese'] = cc.convert(data['chinese'])"""

"#簡轉繁\nfrom opencc import OpenCC\n\ncc = OpenCC('s2t')\n\nfor data in train_data:\n    data['chinese'] = cc.convert(data['chinese'])"

In [77]:
# 讀取資料檔，並將所有單字整理為字典，分別為英文及中文字典，注意，英文為字母的集合，非單字(Word)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for item in train_data:
    input_text = item['english']
    target_text = '\t' + item['chinese'] + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [78]:
print(target_texts[:10])

['\t为了更好的锐度，但是附带的会多一些颗粒度，可以使用这个显影剂的1：1稀释液。\n', '\t他还把宣扬自己思想的所谓《绿皮书》称作“新福音书”。\n', '\t微风推着我去爱抚它的长耳朵\n', '\t它们的先烈们的鲜血是白流了…\n', '\t最后，在1月31日，湖人将前往汽车城底特律挑战活塞队，活塞近来在东部排名第二。\n', '\t“真是天造地设的一对——我父亲喜欢结交名人，杰姬酷爱金钱，”亚历山大在婚礼上讥讽道。他和克里斯蒂娜从未同他们的继母和睦相处过。\n', '\t2006年，沃尔玛的推荐引擎竟将《人猿星球》与马丁·路德·金的记录片配成了一对，为此沃尔玛遭到了种族歧视的指控。\n', '\t通过电子探针显微分析确定贫化渣中主要铜相为冰铜相。\n', '\t吉姆靠给人擦皮鞋为生。\n', '\t用甘氨酸模拟胶原，研究间苯二酚-恶唑烷E鞣性基质的形成以及与胶原之间的反应特性。\n']


In [79]:
# 字典排序            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [80]:
# 計算編碼器、解碼器的最大長度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [81]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 50
Number of unique input tokens: 72
Number of unique output tokens: 733
Max sequence length for inputs: 197
Max sequence length for outputs: 70


In [82]:
# 以dict儲存字典單字及序號
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [83]:
# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [84]:
print(input_token_index)

{' ': 0, '"': 1, '&': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '/': 9, '0': 10, '1': 11, '2': 12, '3': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'R': 37, 'S': 38, 'T': 39, 'U': 40, 'W': 41, 'Y': 42, 'a': 43, 'b': 44, 'c': 45, 'd': 46, 'e': 47, 'f': 48, 'g': 49, 'h': 50, 'i': 51, 'j': 52, 'k': 53, 'l': 54, 'm': 55, 'n': 56, 'o': 57, 'p': 58, 'q': 59, 'r': 60, 's': 61, 't': 62, 'u': 63, 'v': 64, 'w': 65, 'x': 66, 'y': 67, 'z': 68, '—': 69, '“': 70, '…': 71}


In [85]:
print(target_token_index)

{'\t': 0, '\n': 1, ' ': 2, '"': 3, '-': 4, '.': 5, '0': 6, '1': 7, '2': 8, '3': 9, '5': 10, '6': 11, '8': 12, '9': 13, '?': 14, 'A': 15, 'C': 16, 'E': 17, 'I': 18, 'K': 19, 'L': 20, 'O': 21, 'a': 22, 'l': 23, 'm': 24, 'u': 25, '·': 26, '—': 27, '“': 28, '”': 29, '…': 30, '、': 31, '。': 32, '《': 33, '》': 34, '「': 35, '」': 36, '一': 37, '丁': 38, '三': 39, '上': 40, '下': 41, '不': 42, '与': 43, '且': 44, '世': 45, '业': 46, '东': 47, '丝': 48, '两': 49, '严': 50, '个': 51, '中': 52, '丹': 53, '为': 54, '主': 55, '丽': 56, '久': 57, '么': 58, '义': 59, '之': 60, '习': 61, '书': 62, '买': 63, '了': 64, '予': 65, '争': 66, '二': 67, '于': 68, '云': 69, '亚': 70, '些': 71, '亟': 72, '交': 73, '亦': 74, '产': 75, '享': 76, '亮': 77, '亲': 78, '人': 79, '什': 80, '今': 81, '从': 82, '他': 83, '代': 84, '令': 85, '以': 86, '们': 87, '价': 88, '任': 89, '份': 90, '伊': 91, '优': 92, '会': 93, '传': 94, '伤': 95, '伴': 96, '似': 97, '但': 98, '位': 99, '住': 100, '体': 101, '作': 102, '你': 103, '使': 104, '供': 105, '俄': 106, '保': 107, '信': 108, '候': 109, '做': 11

In [86]:
# 設定 encoder_input、decoder_input對應的順序    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [87]:
# 建立 encoder LSTM 隱藏層
origin_encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_0')
origin_encoder = LSTM(latent_dim, return_state=True)
origin_encoder_outputs, origin_state_h, origin_state_c = origin_encoder(origin_encoder_inputs)
origin_encoder_states = [origin_state_h, origin_state_c]

encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_1')
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
print(encoder_outputs)
print(state_h)
print(state_c)
encoder_outputs.type_spec

KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_4/PartitionedCall:0', description="created by layer 'lstm_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_4/PartitionedCall:2', description="created by layer 'lstm_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_4/PartitionedCall:3', description="created by layer 'lstm_4'")


TensorSpec(shape=(None, 256), dtype=tf.float32, name=None)

In [88]:
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

In [89]:
# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='input_2')
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [90]:
# 定義模型，由 encoder_input_data 及 decoder_input_data 轉換為 decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [91]:
# 訓練
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
1/1 [==============================] - 19s 19s/step - loss: 3.1431 - val_loss: 3.9181
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 3.1342 - val_loss: 3.9125
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 3.1222 - val_loss: 3.8097
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 2.9456 - val_loss: 3.8037
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 2.8767 - val_loss: 3.7869
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 2.8574 - val_loss: 3.8119
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 2.8422 - val_loss: 3.7987
Epoch 8/100
1/1 [==============================] - 1s 1s/step - loss: 2.8302 - val_loss: 3.8198
Epoch 9/100
1/1 [==============================] - 1s 1s/step - loss: 2.8341 - val_loss: 3.8100
Epoch 10/100
1/1 [==============================] - 2s 2s/step - loss: 2.8167 - val_loss: 3.8269
Epoch 11/100
1/1 [===================

1/1 [==============================] - 1s 1s/step - loss: 2.7114 - val_loss: 3.8358
Epoch 86/100
1/1 [==============================] - 1s 1s/step - loss: 2.6971 - val_loss: 3.8369
Epoch 87/100
1/1 [==============================] - 1s 1s/step - loss: 2.7039 - val_loss: 3.8348
Epoch 88/100
1/1 [==============================] - 1s 1s/step - loss: 2.6916 - val_loss: 3.8391
Epoch 89/100
1/1 [==============================] - 1s 1s/step - loss: 2.7131 - val_loss: 3.8362
Epoch 90/100
1/1 [==============================] - 1s 1s/step - loss: 2.6908 - val_loss: 3.8319
Epoch 91/100
1/1 [==============================] - 1s 1s/step - loss: 2.7154 - val_loss: 3.8381
Epoch 92/100
1/1 [==============================] - 1s 1s/step - loss: 2.6879 - val_loss: 3.8378
Epoch 93/100
1/1 [==============================] - 1s 1s/step - loss: 2.7010 - val_loss: 3.8360
Epoch 94/100
1/1 [==============================] - 1s 1s/step - loss: 2.6854 - val_loss: 3.8380
Epoch 95/100
1/1 [=========================

In [92]:
# 定義編碼器取樣模型
encoder_model = Model(encoder_inputs, encoder_states)

In [93]:
# 定義解碼器的input
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [94]:
# 定義解碼器 LSTM 模型
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

In [95]:
# 以編碼器的記憶狀態 h 及 c 為解碼器的記憶狀態  
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

print([decoder_inputs] + decoder_states_inputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
'''
decoder_model = Model(inputs=[decoder_inputs].append(decoder_states_inputs), outputs=[decoder_outputs].append(decoder_states))
'''

[<KerasTensor: shape=(None, None, 733) dtype=float32 (created by layer 'input_2')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_3')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_4')>]


'\ndecoder_model = Model(inputs=[decoder_inputs].append(decoder_states_inputs), outputs=[decoder_outputs].append(decoder_states))\n'

In [96]:
# 建立反向的 dict，才能透過查詢將數值轉回文字
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [97]:
# 模型預測，並取得翻譯結果(中文)    
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [103]:
test_texts = []
test_characters = set()
for text in test_data:
    test_texts.append(text['english'])
    for char in text['english']:
        if char not in target_characters:
            test_characters.add(char)

print(test_texts)
print(test_characters)
print(input_characters)

['He had formerly been in business at Bristol, but failed in debt to a number of people, compounded, and went to America.', 'Basing on the on site tests of anchor, authors found that anchors have obvious pre stress loss problem during stretching and locking, analyzed and proposed several solutions.', 'From hair tip first began gradually, after all, through from downward, nodular comb.', 'The sky began to be clear up a bit when we left St Gallen abbey and library.', "Once more, Cinderella's fairy godmother reminded her to be home by midnight.", 'This paper introduces the demand analysis and function design in detail, gives the source codes of relevant interface functions and base algorithm.', 'Manufacturer of thin and ultra-thin non-ferrous metal foils mainly made of copper, copper-alloys, nickel, genuine silver and nickel-silver.', "All day thy wings have fann'd At that far height, the cold thin atmosphere: Yet stoop not, weary, to the welcome land, Though the dark night is near.", 'Th

In [ ]:
# 設定 encoder_input、decoder_input對應的順序    
for i, test_text in enumerate(test_texts):
    for t, char in enumerate(test_text):
        encoder_input_data[i, t, test_token_index[char]] = 1.

In [98]:
# 測試100次
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('*')
    print('Input sentence:', input_texts[seq_index])
    try:
        print('Decoded sentence:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Decoded sentence:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])


*
Input sentence: For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.
Decoded sentence: 这人的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: He calls the Green Book, his book of teachings, “the new gospel.
Decoded sentence: 这人的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: And the light breeze moves me to caress her long ear
Decoded sentence: 这人的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: They have the blood of martyrs is the White to flow …
Decoded sentence: 这人的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: Finally, the Lakers head to the Motor City to take on a Pistons team that currently owns the Eastern Conference's second best record (1/31). L.
Decoded sentence: 这人的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: "The perfect ma

KeyboardInterrupt: 

In [99]:
print(len(encoder_input_data))

50
